In [20]:
# Import dependencies
import sqlalchemy
from sqlalchemy import create_engine, inspect, MetaData
from sqlalchemy import Column, Integer, String, Float, Numeric
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
import pandas as pd
import matplotlib
from matplotlib import style
style.use('seaborn')
import matplotlib.pyplot as plt

In [21]:
!rm -r 'hawaii.sqlite'

rm: hawaii.sqlite: No such file or directory


In [22]:
Base = declarative_base()

In [23]:
# Establish database connection
engine = create_engine('sqlite:///Resources/hawaii.sqlite')

In [24]:
class measurement_table(Base):
    __tablename__= "Measurement"
    id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(String)
    precipitation = Column(Integer)
    tobs = Column(Integer)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [25]:
class station_table(Base):
    __tablename__= "Stations"
    id = Column(Integer, primary_key=True)
    station = Column(String)
    name = Column(String)
    latitude = Column(Integer)
    longitude = Column(Integer)
    elevation = Column(Integer)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [26]:
# Tables only exist in python right now
Base.metadata.tables

immutabledict({'Measurement': Table('Measurement', MetaData(bind=None), Column('id', Integer(), table=<Measurement>, primary_key=True, nullable=False), Column('station', String(), table=<Measurement>), Column('date', String(), table=<Measurement>), Column('precipitation', Integer(), table=<Measurement>), Column('tobs', Integer(), table=<Measurement>), schema=None), 'Stations': Table('Stations', MetaData(bind=None), Column('id', Integer(), table=<Stations>, primary_key=True, nullable=False), Column('station', String(), table=<Stations>), Column('name', String(), table=<Stations>), Column('latitude', Integer(), table=<Stations>), Column('longitude', Integer(), table=<Stations>), Column('elevation', Integer(), table=<Stations>), schema=None)})

In [27]:
# Create the tables in the database
Base.metadata.create_all(engine)

In [28]:
inspector = inspect(engine)
inspector.get_table_names()

['Measurement', 'Stations']

In [29]:
def populate_table(engine, table, csvfile):
    conn = engine.connect()
    df_insert = pd.read_csv(csvfile)
    data = df_insert.to_dict(orient='records')
    conn.execute(table.insert(),data)

In [30]:
populate_table(engine, measurement_table.__table__, "Resources/clean_measurements.csv")

In [31]:
populate_table(engine, station_table.__table__, "Resources/clean_stations.csv")

In [32]:
columns = inspector.get_columns('Measurement')
for column in columns:
    print(column["name"], column["type"])

id INTEGER
station VARCHAR
date VARCHAR
precipitation INTEGER
tobs INTEGER


In [33]:
# Test for the Measurements table
engine.execute('SELECT * FROM Measurement LIMIT 10').fetchall()

[(1, 'USC00519397', '1/1/10', None, 65),
 (2, 'USC00519397', '1/2/10', None, 63),
 (3, 'USC00519397', '1/3/10', None, 74),
 (4, 'USC00519397', '1/4/10', None, 76),
 (5, 'USC00519397', '1/7/10', None, 70),
 (6, 'USC00519397', '1/8/10', None, 64),
 (7, 'USC00519397', '1/9/10', None, 68),
 (8, 'USC00519397', '1/10/10', None, 73),
 (9, 'USC00519397', '1/11/10', None, 64),
 (10, 'USC00519397', '1/12/10', None, 61)]

In [34]:
# Test for the Stations table
engine.execute("SELECT * FROM Stations LIMIT 10").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6),
 (6, 'USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 21.33556, -157.71139, 19.5),
 (7, 'USC00519281', 'WAIHEE 837.5, HI US', 21.45167, -157.84888999999995, 32.9),
 (8, 'USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 21.3152, -157.9992, 0.9),
 (9, 'USC00516128', 'MANOA LYON ARBO 785.2, HI US', 21.3331, -157.8025, 152.4)]